<a href="https://colab.research.google.com/github/AchinthaThennakoon/customer-churn-prediction-using-ANN/blob/main/customer_churn_prediction(with_handling_class_imbalance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split


In [4]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.4 MB/s eta 0:00:00


In [5]:
from tensorflow_addons import losses

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

# Import dataset

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL/customer_churn_prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [9]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Preprocess dataset

In [10]:
#remove customer id
df.drop('customerID',axis='columns',inplace=True)

In [11]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## handle missing values

In [12]:
df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## catogorical columns

In [13]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [14]:
def print_unique_col_values(df):
  for col in df:
    print(f'{col}:{df[col].unique()}')

In [15]:
print_unique_col_values(df)

gender:['Female' 'Male']
SeniorCitizen:[0 1]
Partner:['Yes' 'No']
Dependents:['No' 'Yes']
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService:['No' 'Yes']
MultipleLines:['No phone service' 'No' 'Yes']
InternetService:['DSL' 'Fiber optic' 'No']
OnlineSecurity:['No' 'Yes' 'No internet service']
OnlineBackup:['Yes' 'No' 'No internet service']
DeviceProtection:['No' 'Yes' 'No internet service']
TechSupport:['No' 'Yes' 'No internet service']
StreamingTV:['No' 'Yes' 'No internet service']
StreamingMovies:['No' 'Yes' 'No internet service']
Contract:['Month-to-month' 'One year' 'Two year']
PaperlessBilling:['Yes' 'No']
PaymentMethod:['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:['29.85' '1889

### convert string to numbers in TotalCharges column

add parameter errors='coerce' for convert bad non numeric values to NaN, then check this values by isnull

In [16]:
print (pd.to_numeric(df.TotalCharges, errors='coerce').isnull().sum())

11


iloc = stands for "integer location" and is primarily used for accessing and retrieving data from pandas DataFrame objects using integer-based indexing.

In [17]:
df.iloc[488].TotalCharges

' '

Remove rows with space in TotalCharges

In [18]:
df1 = df[df.TotalCharges!=' ']

In [19]:
print (pd.to_numeric(df1.TotalCharges, errors='coerce').isnull().sum())

0


In [20]:
df.shape

(7043, 20)

In [21]:
df1.shape

(7032, 20)

In [22]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

<ipython-input-22-b67e0c3d31a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.TotalCharges = pd.to_numeric(df1.TotalCharges)


In [23]:
df1.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [24]:
df1.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Replace 'No internet service' witn 'No' & Convert Yes and No to 1 or 0



In [25]:
df1.replace(to_replace='No internet service',value='No',inplace=True)
df1.replace(to_replace='No phone service',value='No',inplace=True)
print_unique_col_values(df1)

gender:['Female' 'Male']
SeniorCitizen:[0 1]
Partner:['Yes' 'No']
Dependents:['No' 'Yes']
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService:['No' 'Yes']
MultipleLines:['No' 'Yes']
InternetService:['DSL' 'Fiber optic' 'No']
OnlineSecurity:['No' 'Yes']
OnlineBackup:['Yes' 'No']
DeviceProtection:['No' 'Yes']
TechSupport:['No' 'Yes']
StreamingTV:['No' 'Yes']
StreamingMovies:['No' 'Yes']
Contract:['Month-to-month' 'One year' 'Two year']
PaperlessBilling:['Yes' 'No']
PaymentMethod:['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn:['No' 'Yes']


<ipython-input-25-c09e22eff650>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace(to_replace='No internet service',value='No',inplace=True)
<ipython-input-25-c09e22eff650>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace(to_replace='No phone service',value='No',inplace=True)


**Convert Yes and No to 1 or 0**

In [26]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

for col in yes_no_columns:
  df1[col].replace({'Yes':1,'No':0},inplace=True)

print_unique_col_values(df1)

<ipython-input-26-18bc0f81cac0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col].replace({'Yes':1,'No':0},inplace=True)


gender:['Female' 'Male']
SeniorCitizen:[0 1]
Partner:[1 0]
Dependents:[0 1]
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService:[0 1]
MultipleLines:[0 1]
InternetService:['DSL' 'Fiber optic' 'No']
OnlineSecurity:[0 1]
OnlineBackup:[1 0]
DeviceProtection:[0 1]
TechSupport:[0 1]
StreamingTV:[0 1]
StreamingMovies:[0 1]
Contract:['Month-to-month' 'One year' 'Two year']
PaperlessBilling:[1 0]
PaymentMethod:['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn:[0 1]


### 'Female':1,'Male':0 in gender column

In [27]:
df1.gender.replace({'Female':1,'Male':0},inplace=True)

<ipython-input-27-56c125f71914>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.gender.replace({'Female':1,'Male':0},inplace=True)


In [28]:
df1.gender.unique()

array([1, 0])

### One hot encoding

In [29]:
print_unique_col_values(df1)

gender:[1 0]
SeniorCitizen:[0 1]
Partner:[1 0]
Dependents:[0 1]
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService:[0 1]
MultipleLines:[0 1]
InternetService:['DSL' 'Fiber optic' 'No']
OnlineSecurity:[0 1]
OnlineBackup:[1 0]
DeviceProtection:[0 1]
TechSupport:[0 1]
StreamingTV:[0 1]
StreamingMovies:[0 1]
Contract:['Month-to-month' 'One year' 'Two year']
PaperlessBilling:[1 0]
PaymentMethod:['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn:[0 1]


In [30]:
df2= pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [31]:
print_unique_col_values(df2)

gender:[1 0]
SeniorCitizen:[0 1]
Partner:[1 0]
Dependents:[0 1]
tenure:[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService:[0 1]
MultipleLines:[0 1]
OnlineSecurity:[0 1]
OnlineBackup:[1 0]
DeviceProtection:[0 1]
TechSupport:[0 1]
StreamingTV:[0 1]
StreamingMovies:[0 1]
PaperlessBilling:[1 0]
MonthlyCharges:[29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges:[  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn:[0 1]
InternetService_DSL:[1 0]
InternetService_Fiber optic:[0 1]
InternetService_No:[0 1]
Contract_Month-to-month:[1 0]
Contract_One year:[0 1]
Contract_Two year:[0 1]
PaymentMethod_Bank transfer (automatic):[0 1]
PaymentMethod_Credit card (automatic):[0 1]
PaymentMethod_Electronic check:[1 0]
PaymentMethod_Mailed check:[0 1]


In [32]:
df2.dtypes

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No             

## Scaling

scale MonthlyCharges,tenure,TotalCharges columns using min max scaler

In [33]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [34]:
print_unique_col_values(df2)

gender:[1 0]
SeniorCitizen:[0 1]
Partner:[1 0]
Dependents:[0 1]
tenure:[0.         0.46478873 0.01408451 0.61971831 0.09859155 0.29577465
 0.12676056 0.38028169 0.85915493 0.16901408 0.21126761 0.8028169
 0.67605634 0.33802817 0.95774648 0.71830986 0.98591549 0.28169014
 0.15492958 0.4084507  0.64788732 1.         0.22535211 0.36619718
 0.05633803 0.63380282 0.14084507 0.97183099 0.87323944 0.5915493
 0.1971831  0.83098592 0.23943662 0.91549296 0.11267606 0.02816901
 0.42253521 0.69014085 0.88732394 0.77464789 0.08450704 0.57746479
 0.47887324 0.66197183 0.3943662  0.90140845 0.52112676 0.94366197
 0.43661972 0.76056338 0.50704225 0.49295775 0.56338028 0.07042254
 0.04225352 0.45070423 0.92957746 0.30985915 0.78873239 0.84507042
 0.18309859 0.26760563 0.73239437 0.54929577 0.81690141 0.32394366
 0.6056338  0.25352113 0.74647887 0.70422535 0.35211268 0.53521127]
PhoneService:[0 1]
MultipleLines:[0 1]
OnlineSecurity:[0 1]
OnlineBackup:[1 0]
DeviceProtection:[0 1]
TechSupport:[0 1]
Stream

# Split train and test data

In [35]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=5)

In [37]:
X_train.shape

(5625, 26)

In [38]:
X_test.shape

(1407, 26)

In [39]:
y_train.head()

5664    1
101     0
2621    0
392     0
1327    1
Name: Churn, dtype: int64

# Build a model (ANN)

In [40]:
def ANN(X_train, y_train, X_test, y_test,loss,weights):
  model = keras.Sequential([
      keras.layers.Dense(26,input_dim=26,activation='relu'),
      keras.layers.Dense(15,activation='relu'),
      keras.layers.Dense(1,activation='sigmoid')
  ])

  model.compile(
      optimizer='adam',
      loss= loss,
      metrics=['accuracy']
  )

  if weights== -1:
    model.fit(X_train, y_train,epochs=100)
  else:
    model.fit(X_train, y_train,epochs=100,class_weight = weights)

  print(model.evaluate( X_test, y_test))

  y_preds = model.predict(X_test)
  y_preds = np.round(y_preds)

  print("Classification Report: \n", classification_report(y_test, y_preds))

  return y_preds

In [41]:
x = np.round(0.6)
x

1.0

# Prediction without handling class imbalance

In [42]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 2s 4ms/step - loss: 0.4862 - accuracy: 0.7639
Epoch 2/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4229 - accuracy: 0.7957
Epoch 3/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4169 - accuracy: 0.8039
Epoch 4/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4152 - accuracy: 0.8028
Epoch 5/100
176/176 [==============================] - 1s 7ms/step - loss: 0.4131 - accuracy: 0.8037
Epoch 6/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4102 - accuracy: 0.8046
Epoch 7/100
176/176 [==============================] - 1s 5ms/step - loss: 0.4092 - accuracy: 0.8062
Epoch 8/100
176/176 [==============================] - 1s 5ms/step - loss: 0.4076 - accuracy: 0.8084
Epoch 9/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4054 - accuracy: 0.8101
Epoch 10/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4042 - accura

In [43]:
y_train.value_counts()

0    4164
1    1461
Name: Churn, dtype: int64

In [44]:
y_test.value_counts()

0    999
1    408
Name: Churn, dtype: int64

# Handling class imbalance

## Method 1: Undersampling

In [45]:
df1.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,0,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0
2,0,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,0,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,1,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [46]:
df2.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,0.000000,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0.014085,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [47]:
count_class_0,count_class_1 = df2.Churn.value_counts()

In [48]:
count_class_0,count_class_1

(5163, 1869)

In [49]:
# Divide by class
df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [50]:
df_class_0_under = df_class_0.sample(count_class_1)
df_balanced_under = pd.concat([df_class_0_under,df_class_1],axis=0)

print('Random under-sampling:')
print(df_balanced_under.Churn.value_counts())

Random under-sampling:
0    1869
1    1869
Name: Churn, dtype: int64


In [51]:
x = df_balanced_under.drop('Churn',axis='columns')
y = df_balanced_under['Churn']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=15, stratify=y)

In [52]:
y_train.value_counts()

0    1495
1    1495
Name: Churn, dtype: int64

In [53]:
y_preds = ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
94/94 [==============================] - 2s 4ms/step - loss: 0.6354 - accuracy: 0.6405
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 0.5300 - accuracy: 0.7525
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4983 - accuracy: 0.7562
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4912 - accuracy: 0.7572
Epoch 5/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4858 - accuracy: 0.7625
Epoch 6/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4833 - accuracy: 0.7605
Epoch 7/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4816 - accuracy: 0.7702
Epoch 8/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4809 - accuracy: 0.7676
Epoch 9/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4780 - accuracy: 0.7669
Epoch 10/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4762 - accuracy: 0.7676
Epoch 11/

## Method2: Oversampling


### random oversampling

In [54]:
df_class_1_over = df2.sample(count_class_0)
df_oversampled = pd.concat([df_class_1_over,df_class_0],axis=0)

print('Random over-sampling:')
print(df_oversampled.Churn.value_counts())

Random over-sampling:
0    8973
1    1353
Name: Churn, dtype: int64


In [55]:
X = df_oversampled.drop('Churn',axis='columns')
y = df_oversampled['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [56]:
# Number of classes in training Data
y_train.value_counts()

0    7178
1    1082
Name: Churn, dtype: int64

In [57]:
y_pred = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.3671 - accuracy: 0.8487
Epoch 2/100
259/259 [==============================] - 1s 2ms/step - loss: 0.2989 - accuracy: 0.8751
Epoch 3/100
259/259 [==============================] - 1s 2ms/step - loss: 0.2944 - accuracy: 0.8746
Epoch 4/100
259/259 [==============================] - 1s 3ms/step - loss: 0.2926 - accuracy: 0.8762
Epoch 5/100
259/259 [==============================] - 1s 5ms/step - loss: 0.2911 - accuracy: 0.8763
Epoch 6/100
259/259 [==============================] - 1s 5ms/step - loss: 0.2898 - accuracy: 0.8800
Epoch 7/100
259/259 [==============================] - 1s 3ms/step - loss: 0.2889 - accuracy: 0.8753
Epoch 8/100
259/259 [==============================] - 1s 3ms/step - loss: 0.2887 - accuracy: 0.8788
Epoch 9/100
259/259 [==============================] - 1s 3ms/step - loss: 0.2873 - accuracy: 0.8808
Epoch 10/100
259/259 [==============================] - 1s 3ms/step - loss: 0.2871 - accura

## Method3: SMOTE

In [58]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [59]:
from  imblearn.over_sampling import SMOTE

In [60]:
smote = SMOTE(sampling_strategy='minority')
X_sm,y_sm = smote.fit_resample(X,y)
y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [62]:
# Number of classes in training Data
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [63]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 2s 4ms/step - loss: 0.5254 - accuracy: 0.7397
Epoch 2/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4703 - accuracy: 0.7777
Epoch 3/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4603 - accuracy: 0.7815
Epoch 4/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4551 - accuracy: 0.7831
Epoch 5/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4508 - accuracy: 0.7864
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4463 - accuracy: 0.7892
Epoch 7/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4436 - accuracy: 0.7921
Epoch 8/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4398 - accuracy: 0.7916
Epoch 9/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4363 - accuracy: 0.7956
Epoch 10/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4326 - accura

## Method4: Use of Ensemble with undersampling


In [64]:
df2.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [65]:
# Regain Original features and labels
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [67]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [68]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [69]:
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
684,1,0,0,0,0.000000,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2446,1,0,0,0,0.239437,1,1,0,1,0,...,1,0,1,0,0,0,1,0,0,1
1680,0,0,1,1,0.774648,1,1,0,0,0,...,0,1,0,1,0,0,0,0,1,0
2220,0,0,1,0,1.000000,1,0,1,1,0,...,0,0,0,0,1,1,0,0,0,0
2842,1,0,0,0,0.042254,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,0


In [70]:
df3_class0 = df3[df3.Churn==0]
df3_class1 = df3[df3.Churn==1]

In [71]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis='columns')
    y_train = df_train.Churn
    return X_train, y_train

In [72]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 2s 4ms/step - loss: 0.6699 - accuracy: 0.5736
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 0.5493 - accuracy: 0.7505
Epoch 3/100
94/94 [==============================] - 0s 5ms/step - loss: 0.5024 - accuracy: 0.7625
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7639
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4898 - accuracy: 0.7676
Epoch 6/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4845 - accuracy: 0.7672
Epoch 7/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4834 - accuracy: 0.7739
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.7742
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4780 - accuracy: 0.7773
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4771 - accuracy: 0.7766
Epoch 11/

In [74]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 4ms/step - loss: 0.6423 - accuracy: 0.6261
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 0.5274 - accuracy: 0.7495
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4981 - accuracy: 0.7589
Epoch 4/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4908 - accuracy: 0.7602
Epoch 5/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4895 - accuracy: 0.7602
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4850 - accuracy: 0.7625
Epoch 7/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4834 - accuracy: 0.7666
Epoch 8/100
94/94 [==============================] - 0s 5ms/step - loss: 0.4811 - accuracy: 0.7679
Epoch 9/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4796 - accuracy: 0.7635
Epoch 10/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4786 - accuracy: 0.7615
Epoch 11/

In [73]:
X_train, y_train = get_train_batch(df3_class0, df3_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
83/83 [==============================] - 1s 2ms/step - loss: 0.6429 - accuracy: 0.6285
Epoch 2/100
83/83 [==============================] - 0s 3ms/step - loss: 0.5232 - accuracy: 0.7586
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4935 - accuracy: 0.7693
Epoch 4/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4823 - accuracy: 0.7715
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4774 - accuracy: 0.7750
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4732 - accuracy: 0.7784
Epoch 7/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4704 - accuracy: 0.7791
Epoch 8/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4672 - accuracy: 0.7833
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.7806
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4628 - accuracy: 0.7814
Epoch 11/

In [75]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones>1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [76]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1033
           1       0.49      0.80      0.61       374

    accuracy                           0.72      1407
   macro avg       0.70      0.75      0.70      1407
weighted avg       0.80      0.72      0.74      1407

